In [8]:
import torch
import torchaudio
import cv2
import seaborn as sns
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image

# Подготовка данных

In [3]:
# Просмотр данных

flowers_dir = r"data/flowers"

all_image_paths = []

for folder in os.listdir(flowers_dir):
    folder_path = os.path.join(flowers_dir, folder)
    if os.path.isdir(folder_path):
        all_image_paths += [os.path.join(folder_path, fname)
                            for fname in os.listdir(folder_path)
                            if fname.endswith(".jpg")]
        
random.shuffle(all_image_paths)

In [4]:
# Дробим сет данных 90 процентов библиотека в 10 процентах запросы
split_index = int(0.9 * len(all_image_paths))
library_paths = all_image_paths[:split_index]
query_paths = all_image_paths[split_index:split_index + 5]  # 5 случайных для визуализации

In [5]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],  # стандарт для ImageNet
                                                     std=[0.229, 0.224, 0.225])])

# Обучение

In [6]:
# Выбираем устройство на котором будет обучаться модел, в моём случае это CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Загрузка ResNet50 без классификатора
base_model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(base_model.children())[:-1])  # удаляем классификатор
model = model.to(device).eval()

d:\A_projects\Test_Task\Flower_Recognizer\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\A_projects\Test_Task\Flower_Recognizer\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\kolyagin/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [9]:
model_path = r"app\weight\resnet50_finder_.pth"
# Сохранение модели
torch.save(model.state_dict(), model_path)
print(f"Модель сохранена в {model_path}")

Модель сохранена в app\weight\resnet50_finder_.pth


In [ ]:
# Функция для получение эмбеддингов
def get_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        embedding = model(image_tensor).squeeze().cpu().numpy()
    return embedding.flatten()

In [ ]:
# Получение эмбедингов библиотеки
library_embeding = []

# Тестирование 

In [ ]:
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
        
accuracy = np.mean(np.array(y_true) == np.array(y_pred))

print(f"Точность на тестовых данных:{accuracy*100:.2f}%")

Посмотрим матрицу ошибок

In [ ]:
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_name)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Матрица ошибок")
plt.tight_layout()
plt.show()
    

In [ ]:
# тестовая функция для отображения 
def predict_image(input_image_path:str):
    